# Price movement prediction with DL
In this notebook we are going to see how to build a deep neural network to predict price movement using LOB data. We are going to cover four main parts:
1. Data download and initial setup.
2. Dataset: input and output.
3. Model architecture.
4. Training pipeline.

For the I/O and the model architecture, we are going to use PyTorch, while for the training pipeline we are going to use PyTorch Lightning.

In [ ]:
import os
from typing import Any, Dict, Optional, Tuple

import numpy as np
import pytorch_lightning as pl
import torch
import torchmetrics
from numpy.lib.stride_tricks import sliding_window_view
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch import nn
from torch.utils.data import DataLoader, Dataset

## Data download and initial setup
The data can be downloaded from the [original website](https://etsin.fairdata.fi/dataset/73eb48d7-4dbc-4a10-a52a-da745b47a649), or a compressed version from [here](https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip).

In this notebook, the latter has been used, then the data is decompressed, and the files are loaded using NumPy. The first 7 days of the FI-2010 dataset are used as training data, and the remaining 3 days are used as test. Finally, the training data is split using a 80/20 ratio respectively for the training and validation sets.

In [50]:
if not all(
    os.path.isfile(path) for path in ["data/train.txt", "data/val.txt", "test.txt"]
):
    # data paths
    train_path = "data/Train_Dst_NoAuction_DecPre_CF_7.txt"
    test_paths = [
        "data/Test_Dst_NoAuction_DecPre_CF_7.txt",
        "data/Test_Dst_NoAuction_DecPre_CF_8.txt",
        "data/Test_Dst_NoAuction_DecPre_CF_9.txt",
    ]

    # download data
    if not os.path.isfile("data/data.zip"):
        !wget "https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip" -P data/
        !unzip -n data/data.zip -d data/

    # load training + validation data
    train_val_data = np.loadtxt(train_path, unpack=True)

    # split into train and validation
    train_slice = slice(0, int(0.8 * train_val_data.shape[0]))
    val_slice = slice(int(0.8 * train_val_data.shape[0]), train_val_data.shape[0])

    train_data = train_val_data[train_slice, :]
    val_data = train_val_data[val_slice, :]

    # load test data
    test_data = np.concatenate([np.loadtxt(path, unpack=True) for path in test_paths])

    # save train, val, test data to single
    np.savetxt("data/train.txt", train_data.T)
    np.savetxt("data/val.txt", val_data.T)
    np.savetxt("data/test.txt", test_data.T)

else:
    # data paths
    train_path = "data/train.txt"
    val_path = "data/val.txt"
    test_path = "data/test.txt"

    # load train, val, test data
    train_data = np.loadtxt(train_path)
    val_data = np.loadtxt(val_path)
    test_data = np.loadtxt(test_path)

--2021-12-07 20:40:55--  https://raw.githubusercontent.com/zcakhaa/DeepLOB-Deep-Convolutional-Neural-Networks-for-Limit-Order-Books/master/data/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56278154 (54M) [application/zip]
Saving to: ‘data/data.zip’

data.zip            100%[===================>]  53.67M   167MB/s    in 0.3s    

2021-12-07 20:40:56 (167 MB/s) - ‘data/data.zip’ saved [56278154/56278154]

Archive:  data/data.zip
  inflating: data/Test_Dst_NoAuction_DecPre_CF_7.txt  
  inflating: data/Test_Dst_NoAuction_DecPre_CF_9.txt  
  inflating: data/Test_Dst_NoAuction_DecPre_CF_8.txt  
  inflating: data/Train_Dst_NoAuction_DecPre_CF_7.txt  


In [51]:
train_data.shape, val_data.shape, test_data.shape

((203800, 149), (50950, 149), (139587, 149))

## Dataset: input and output
The `Dataset` class in PyTorch is an abstract class representing a dataset. It is used to define what's the input that will be fed to the model, and if the problem is supervised, the labels.

The dataset class should override two methods:
* `__len__`: so that `len(dataset)` returns the size of the dataset.
* `__getitem__`: to support the indexing, such that `dataset[i]` can be used to get the *i*-th sample.

In our case, each sample will be a dictionary `{"input": ..., "labels": ...}` containing both the input (i.e., window of 100 trades starting at time step *i*) and the label (i.e., price movement computed using the smoothing labelling strategy defined in the slides).

In [ ]:
def sliding_window_data(
    input_data: np.ndarray, labels: np.ndarray, window_length: int
) -> Tuple[np.ndarray, np.ndarray]:
    input_data = np.array(input_data)
    labels = np.array(labels)
    input_windows = sliding_window_view(input_data, window_length, axis=0).transpose(
        (0, 2, 1)
    )
    labels_trimmed = labels[window_length - 1 :]
    return input_windows, labels_trimmed


def preprocess(
    data: np.ndarray, window_length: int, prediction_horizon_index: int
) -> Tuple[torch.Tensor, torch.Tensor]:
    data_copy = data.copy()
    # As input, we select the first 40 columns. These are the first 10 levels
    # of the orderbook, containing price and volume for both bid and ask
    input_data = data_copy[:, :40]
    # As labels, we select the last 5 columns of the orderbook.
    # The labels are (1, 2, 3), which respectively represent
    # (positive percentage change, stationary, negative percentage change).
    labels = data_copy[:, -5:]
    # Make the labels start from 0
    labels -= 1

    # Each of the 5 column of the labels represents
    # a different projection horizon (i.e., 1, 2, 3, 5, 10).
    # We keep just one of those
    labels = labels[:, prediction_horizon_index]

    # Split the input data in windows of length `window_length`,
    # and trim the first `window_length` elements of the labels
    input_windows, labels_trimmed = sliding_window_data(
        input_data, labels, window_length
    )

    # Cast np arrays into tensors and add one dimension to input to account for convolutions
    input_windows = torch.tensor(input_windows, dtype=torch.float).unsqueeze(1)
    labels = torch.tensor(labels_trimmed, dtype=torch.long)

    return input_windows, labels

In [52]:
class LobDataset(Dataset):
    def __init__(
        self,
        data: np.ndarray,
        window_length: int = 100,
        prediction_horizon_index: int = 4,
    ) -> None:
        super(LobDataset, self).__init__()
        self.input_windows, self.labels = preprocess(
            data, window_length, prediction_horizon_index
        )

    def __len__(self) -> int:
        return self.input_windows.shape[0]

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        return {"input": self.input_windows[idx], "labels": self.labels[idx]}

# Model
The `nn.Module` class is the base class for all neural network modules. To define our model, we need to instantiate the architecture in the `__init__` method, and we'll need to override the `forward` method, in which we'll have to define the forward step of our neural network.

In [53]:
class ConvolutionBlock(nn.Module):
    def __init__(
        self, in_channels: int, out_channels: int, activation_function: nn.Module
    ) -> None:
        super(ConvolutionBlock, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, (1, 2), (1, 2)),
            activation_function,
            nn.BatchNorm2d(out_channels),
            activation_function,
            nn.Conv2d(out_channels, out_channels, (4, 1)),
            nn.BatchNorm2d(out_channels),
            activation_function,
            nn.Conv2d(out_channels, out_channels, (4, 1)),
            activation_function,
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.conv_block(x)


class InceptionBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int) -> None:
        super(InceptionBlock, self).__init__()

        self.block1 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, (1, 1), padding="same"),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, (3, 1), padding="same"),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(out_channels),
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, (1, 1), padding="same"),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, (5, 1), padding="same"),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(out_channels),
        )
        self.block3 = nn.Sequential(
            nn.MaxPool2d((3, 1), stride=(1, 1), padding=(1, 0)),
            nn.Conv2d(in_channels, out_channels, (1, 1), padding="same"),
            nn.LeakyReLU(0.01),
            nn.BatchNorm2d(out_channels),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out_1 = self.block1(x)
        out_2 = self.block2(x)
        out_3 = self.block3(x)

        return torch.cat((out_1, out_2, out_3), dim=1)


class DeepLOB(nn.Module):
    def __init__(self, num_classes: int = 3) -> None:
        super(DeepLOB, self).__init__()

        # convolution blocks
        self.conv_block1 = ConvolutionBlock(1, 32, activation_function=nn.LeakyReLU())
        self.conv_block2 = ConvolutionBlock(32, 32, activation_function=nn.Tanh())

        # convolution block 3 is not standard
        self.conv_block3 = nn.Sequential(
            nn.Conv2d(32, 32, (1, 10)),
            nn.LeakyReLU(0.01),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
            nn.Conv2d(32, 32, (4, 1)),
            nn.LeakyReLU(0.01),
        )

        # inception block
        self.inception_block = InceptionBlock(32, 64)

        # lstm layer + linear to output logits
        self.lstm = nn.LSTM(input_size=192, hidden_size=64, batch_first=True)
        self.fc1 = nn.Linear(64, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # conv blocks
        out = self.conv_block1(x)
        out = self.conv_block2(out)
        out = self.conv_block3(out)

        # inception block
        out = self.inception_block(out)

        # reshape data to feed lstm
        out = out.permute(0, 2, 1, 3)
        out = out.reshape(out.shape[0], -1, out.shape[2])

        # use lstm and take last item
        out, _ = self.lstm(out)
        out = out[:, -1, :]

        # return softmaxed classes
        return torch.softmax(self.fc1(out), dim=1)

## Training pipeline
For the training pipeline itself, we are going to use PyTorch Lightning. PyTorch Lightning is a high-level programming layer built on top of PyTorch. It makes building and training models faster, easier, and more reliable. With Pytorch Lightning, there is no need to write all the boilerplate code for the backward pass, optimization step, and to define the training and validation loop. There is also quite a number of callbacks defined to use things like early stopping, model checkpoint saving and loading, etc.


<p align="center">
  <img src="https://miro.medium.com/max/700/1*nYwSqPWq4PPnAB4hw5n7gg.png" />
</p>

### PyTorch Lightning data module

In [54]:
class LobDataModule(pl.LightningDataModule):
    def __init__(
        self,
        train_data: np.ndarray,
        val_data: np.ndarray,
        test_data: np.ndarray,
        batch_size: int,
    ) -> None:
        super(LobDataModule, self).__init__()
        self.train_data = train_data
        self.val_data = val_data
        self.test_data = test_data
        self.batch_size = batch_size

    def setup(self, stage: Optional[str] = None) -> None:
        if stage in [None, "fit"]:
            self.trainset = LobDataset(self.train_data)
            self.devset = LobDataset(self.val_data)
        if stage in [None, "test"]:
            self.testset = LobDataset(self.test_data)

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.trainset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=8,
            pin_memory=True,
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.devset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=8,
            pin_memory=True,
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            self.testset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=8,
            pin_memory=True,
        )

### PyTorch Lightning model module

In [55]:
class DeepLobModel(pl.LightningModule):
    def __init__(self, hparams: Dict[str, Any]) -> None:
        super(DeepLobModel, self).__init__()
        self.save_hyperparameters(hparams)
        # model and criterion
        self.model = DeepLOB(self.hparams.num_classes)
        self.criterion = nn.CrossEntropyLoss()

        # metrics to track
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        self.train_f1 = torchmetrics.F1()
        self.val_f1 = torchmetrics.F1()
        self.test_acc = torchmetrics.Accuracy()
        self.test_f1 = torchmetrics.F1()

    def forward(self, batch: torch.Tensor) -> torch.Tensor:
        return self.model(batch["input"])

    def step(self, batch: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
        labels = batch["labels"].view(-1)
        logits = self(batch)
        predictions = torch.argmax(logits, dim=1)
        loss = self.criterion(logits, labels)

        return {"loss": loss, "predictions": predictions}

    def training_step(
        self, batch: Dict[str, torch.Tensor], batch_idx: Optional[int]
    ) -> torch.Tensor:
        step_output = self.step(batch)
        accuracy = self.train_acc(step_output["predictions"], batch["labels"])
        f1_score = self.train_f1(step_output["predictions"], batch["labels"])

        self.log_dict(
            {
                "train_loss": step_output["loss"],
                "train_acc": accuracy,
                "train_f1": f1_score,
            },
            prog_bar=True,
            on_step=True,
            on_epoch=True,
        )

        return step_output["loss"]

    def validation_step(
        self, batch: Dict[str, torch.Tensor], batch_idx: Optional[int]
    ) -> None:
        step_output = self.step(batch)
        accuracy = self.val_acc(step_output["predictions"], batch["labels"])
        f1_score = self.val_f1(step_output["predictions"], batch["labels"])

        self.log_dict(
            {"val_loss": step_output["loss"], "val_acc": accuracy, "val_f1": f1_score},
            prog_bar=True,
            on_step=True,
            on_epoch=True,
        )

    def test_step(
        self, batch: Dict[str, torch.Tensor], batch_idx: Optional[int]
    ) -> None:
        step_output = self.step(batch)
        accuracy = self.val_acc(step_output["predictions"], batch["labels"])
        f1_score = self.val_f1(step_output["predictions"], batch["labels"])
        self.log_dict(
            {
                "test_loss": step_output["loss"],
                "test_acc": accuracy,
                "test_f1": f1_score,
            },
            prog_bar=True,
            on_step=True,
            on_epoch=True,
        )

    def configure_optimizers(self) -> torch.optim.Optimizer:
        return torch.optim.Adam(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay,
        )

## Training

In [56]:
pl.seed_everything(42, workers=True)

hparams = {
    "lr": 0.0001,
    "weight_decay": 0.0,
    "num_classes": 3,
    "batch_size": 128,
}
model = DeepLobModel(hparams)

datamodule = LobDataModule(train_data, val_data, test_data, hparams["batch_size"])

wandb_logger = WandbLogger(offline=False, project="DeepLOB-ai4t")

early_stop_callback = EarlyStopping(
    monitor="val_acc",
    patience=20,
    verbose=False,
    mode="max",
)

checkpoint_callback = ModelCheckpoint(
    dirpath="./saved_checkpoints",
    filename="{epoch}_{val_acc:.3f}",
    monitor="val_acc",
    save_top_k=2,
    save_last=True,
    mode="max",
)

trainer = pl.Trainer(
    gpus=1,
    val_check_interval=1.0,
    max_epochs=150,
    num_sanity_val_steps=0,
    logger=wandb_logger,
    callbacks=[early_stop_callback, checkpoint_callback],
    log_every_n_steps=1,
)
trainer.fit(model=model, datamodule=datamodule)

# wandb.finish()

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DeepLOB          | 143 K 
1 | criterion | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | val_acc   | Accuracy         | 0     
4 | train_f1  | F1               | 0     
5 | val_f1    | F1               | 0     
6 | test_acc  | Accuracy         | 0     
7 | test_f1   | F1               | 0     
-----------------------------------------------
143 K     Trainable params
0         Non-trainable params
143 K     Total params
0.575     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoi

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_acc_epoch,▁▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
train_acc_step,▁▂▃▃▄▃▁▅▄▅▅▅▆▆▅▅▇▅▇▆▆▆▅▆▆▄▅▆▇█▆▇▇▇▅██▇▆▆
train_f1_epoch,▁▄▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇██████████████
train_f1_step,▁▂▃▃▄▃▁▅▄▅▅▅▆▆▅▅▇▅▇▆▆▆▅▆▆▄▅▆▇█▆▇▇▇▅██▇▆▆
train_loss_epoch,█▆▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
train_loss_step,█▇▆▅▅▅█▄▄▄▄▄▃▄▄▄▂▄▃▃▃▃▄▃▃▅▄▃▂▁▃▂▂▂▄▁▁▂▃▃
trainer/global_step,▁▁▁▂▂▂▂▂▂▁▁▃▃▂▂▄▄▄▄▄▅▅▅▅▂▂▆▆▆▆▆▇▇▇▇▇███▃
val_acc_epoch,▁▃▁▂▄▇▇▅▇▇▄▇█▇▇▇██▇████▇██████▇███▇████▇
val_acc_step,▃▃▅▄▅▁▅▅▆▅▃▁▃▅█▇▆▆▇▆▅▅▆▅▁▄▅▅▅▅▅▆▆▇▅▆▅▆▇▇
val_f1_epoch,▁▃▁▂▄▇▇▅▇▇▄▇█▇▇▇██▇████▇██████▇███▇████▇


## Testing

In [ ]:
model = DeepLobModel.load_from_checkpoint(
    "saved_checkpoints/epoch=29_val_acc=0.655.ckpt"
)
test_dataloader = DataLoader(LobDataset(test_data), batch_size=128, shuffle=False)

trainer = pl.Trainer(gpus=1)

trainer.test(model=model, dataloaders=test_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:112: UserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.7377480268478394,
 'test_acc_epoch': 0.7377480268478394,
 'test_f1': 0.7377480268478394,
 'test_f1_epoch': 0.7377480268478394,
 'test_loss': 0.8061350584030151,
 'test_loss_epoch': 0.8061350584030151}
--------------------------------------------------------------------------------


[{'test_acc': 0.7377480268478394,
  'test_acc_epoch': 0.7377480268478394,
  'test_f1': 0.7377480268478394,
  'test_f1_epoch': 0.7377480268478394,
  'test_loss': 0.8061350584030151,
  'test_loss_epoch': 0.8061350584030151}]